In [6]:
# -*- coding: utf-8 -*-
"""UAS_Penalaran_Komputer_Tahap_4.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1B_YOUR_NOTEBOOK_ID_HERE
"""

# Instalasi Library
print("Menginstal library yang dibutuhkan untuk Tahap 4...")
!pip install pandas joblib transformers sentence-transformers numpy > /dev/null 2>&1
print("Instalasi library selesai.")

# Koneksi Google Drive
print("Menghubungkan Google Drive...")
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive terhubung.")

Menginstal library yang dibutuhkan untuk Tahap 4...
Instalasi library selesai.
Menghubungkan Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive terhubung.


In [7]:
import pandas as pd
import os
import joblib # Untuk memuat model TF-IDF
import numpy as np # Untuk memuat BERT embeddings
from sentence_transformers import SentenceTransformer # Untuk model embedding BERT
from sklearn.feature_extraction.text import TfidfVectorizer # Untuk vectorizer TF-IDF
from sklearn.metrics.pairwise import cosine_similarity
import re # Untuk clean_text
from typing import List, Dict, Any, Union, Tuple

# --- Fungsi untuk membuat jalur (diulang agar tersedia di Tahap 4) ---
def create_path(folder_name: str) -> str:
    path = os.path.join('/content/drive/MyDrive/', folder_name)
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Folder '{path}' berhasil dibuat di Google Drive.")
    else:
        print(f"Folder '{path}' sudah ada di Google Drive.")
    return path

# --- Fungsi clean_text (diulang agar tersedia di Tahap 4) ---
def clean_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = re.sub(r'M a h ka m a h A g u n g R e p u blik In d o n esia\n', '', text)
    text = re.sub(r'Disclaimer\n', '', text)
    text = re.sub(r'Kepaniteraan Mahkamah Agung Republik Indonesia berusaha untuk selalu mencantumkan informasi paling kini dan akurat sebagai bentuk komitmen Mahkamah Agung untuk pelayanan publik, transparansi dan akuntabilitas\n', '', text)
    text = re.sub(r'pelaksanaan fungsi peradilan\. Namun dalam hal-hal tertentu masih dimungkinkan terjadi permasalahan teknis terkait dengan akurasi dan keterkinian informasi yang kami sajikan, hal mana akan terus kami perbaiki dari waktu kewaktu\.\n', '', text)
    text = re.sub(r'Dalam hal Anda menemukan inakurasi informasi yang termuat pada situs ini atau informasi yang seharusnya ada, namun belum tersedia, maka harap segera hubungi Kepaniteraan Mahkamah Agung RI melalui :\n', '', text)
    text = re.sub(r'Email : kepaniteraan@mahkamahagung\.go\.id\s+Telp : 021-384 3348 \(ext\.318\)\n', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# --- Fungsi untuk memuat cases.csv (diulang agar tersedia di Tahap 4) ---
def load_cases_data(processed_data_folder: str) -> pd.DataFrame:
    cases_csv_path = os.path.join(processed_data_folder, 'cases.csv')
    print(f"Mencoba memuat cases.csv dari: {cases_csv_path}")
    if not os.path.exists(cases_csv_path):
        print(f"ERROR: File cases.csv TIDAK DITEMUKAN di '{cases_csv_path}'. Pastikan Tahap 2 sudah selesai dengan benar.")
        return pd.DataFrame()
    try:
        df = pd.read_csv(cases_csv_path)
        if df.empty:
            print(f"Peringatan: File cases.csv ditemukan tetapi KOSONG.")
        else:
            print(f"Berhasil memuat {len(df)} kasus dari cases.csv.")
            for col in ['judul', 'ringkasan_fakta', 'argumen_hukum_utama', 'text_pdf', 'amar', 'amar_lainnya', 'catatan_amar']:
                if col in df.columns:
                    df[col] = df[col].astype(str).fillna('')
        return df
    except Exception as e:
        print(f"ERROR: Gagal memuat cases.csv: {e}")
        return pd.DataFrame()

# --- Load Model dan Vektor dari Tahap 3 ---
# Konfigurasi Jalur
base_drive_path = '/content/drive/MyDrive/CBR_Data'
models_output_folder = os.path.join(base_drive_path, 'models')
processed_data_folder = os.path.join(base_drive_path, 'data/processed') # Untuk load cases.csv

# Inisialisasi variabel untuk Tahap 4
loaded_vectorizer = None # TfidfVectorizer jika TF-IDF
loaded_embedding_model = None # SentenceTransformer jika BERT
loaded_case_vectors = None # Numpy array atau sparse matrix dari kasus
loaded_vectorizer_type = None # "TF-IDF" atau "BERT_Embedding"

print("Memuat model dan vektor dari Tahap 3...")
try:
    with open(os.path.join(models_output_folder, 'vectorizer_type.txt'), 'r') as f:
        loaded_vectorizer_type = f.read().strip()
    print(f"Tipe vectorizer yang digunakan: {loaded_vectorizer_type}")

    if loaded_vectorizer_type == "TF-IDF":
        loaded_vectorizer = joblib.load(os.path.join(models_output_folder, 'tfidf_vectorizer.joblib'))
        loaded_case_vectors = joblib.load(os.path.join(models_output_folder, 'tfidf_case_vectors.joblib'))
        print("TF-IDF vectorizer dan case vectors berhasil dimuat.")
    elif loaded_vectorizer_type == "BERT_Embedding":
        print("Memuat ulang SentenceTransformer model untuk BERT Embedding...")
        loaded_embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
        loaded_case_vectors = np.load(os.path.join(models_output_folder, 'bert_case_vectors.npy'))
        print("BERT embedding model dan case vectors berhasil dimuat.")
    else:
        print("ERROR: Tipe vectorizer tidak dikenal.")

except FileNotFoundError:
    print("ERROR: File model Tahap 3 tidak ditemukan. Pastikan Tahap 3 sudah selesai dan menyimpan outputnya.")
except Exception as e:
    print(f"ERROR: Gagal memuat model/vektor dari Tahap 3: {e}")

# Muat df_cases untuk Tahap 4 (juga digunakan oleh fungsi retrieve di bawah)
df_cases = load_cases_data(processed_data_folder)

# --- Definisi Ulang Fungsi retrieve dari Tahap 3 agar bisa digunakan di sini ---
# Ini menggunakan variabel global yang baru saja dimuat: loaded_vectorizer, loaded_embedding_model, etc.
def retrieve(query: str, k: int = 5) -> List[Dict[str, Any]]:
    """
    Menemukan top-k kasus lama yang paling mirip dengan query kasus baru.
    """
    print(f"\nMelakukan retrieval untuk query: '{query[:50]}...' (top-k={k})")

    processed_query = clean_text(query)
    # print(f"DEBUG: Query setelah pre-processing: '{processed_query[:50]}...'")

    query_vector = None
    if loaded_vectorizer_type == "TF-IDF":
        if loaded_vectorizer is not None:
            query_vector = loaded_vectorizer.transform([processed_query])
        else:
            print("ERROR: TF-IDF Vectorizer tidak dimuat.")
            return []
    elif loaded_vectorizer_type == "BERT_Embedding":
        if loaded_embedding_model is not None:
            query_vector = loaded_embedding_model.encode([processed_query], convert_to_tensor=True).cpu().numpy()
        else:
            print("ERROR: BERT Embedding Model tidak dimuat.")
            return []
    else:
        print("ERROR: Tipe vectorizer tidak dikenal atau belum dimuat.")
        return []

    if query_vector is None or loaded_case_vectors is None:
        print("ERROR: Query vector atau loaded_case_vectors kosong. Tidak bisa menghitung kemiripan.")
        return []

    try:
        import scipy.sparse # Import lokal di sini untuk fungsi ini
    except ImportError:
        print("Peringatan: scipy.sparse tidak dapat diimpor. Mungkin tidak diperlukan jika tidak menggunakan TF-IDF.")
        pass # Lanjutkan jika tidak pakai TF-IDF

    # Periksa tipe loaded_case_vectors dengan lebih aman
    is_sparse_matrix = False
    try:
        # Coba akses scipy.sparse.csr_matrix, jika scipy tidak ada, ini akan skip
        if 'scipy.sparse' in globals() and hasattr(scipy.sparse, 'csr_matrix'):
            is_sparse_matrix = isinstance(loaded_case_vectors, scipy.sparse.csr_matrix)
    except NameError: # Jika scipy belum diimport, atau bukan sparse matrix
        pass

    if not isinstance(loaded_case_vectors, np.ndarray) and not is_sparse_matrix:
         print(f"ERROR: loaded_case_vectors memiliki tipe yang tidak didukung untuk similarity: {type(loaded_case_vectors)}")
         return []

    try:
        similarities = cosine_similarity(query_vector, loaded_case_vectors).flatten()
        # print(f"DEBUG: Similarities shape: {similarities.shape}")
    except Exception as e:
        print(f"ERROR: Gagal menghitung cosine similarity: {e}")
        print(f"  Query vector shape: {query_vector.shape if query_vector is not None else 'None'}")
        print(f"  Corpus vectors shape: {loaded_case_vectors.shape if loaded_case_vectors is not None else 'None'}")
        return []

    if len(similarities) < k:
        print(f"Peringatan: Jumlah sampel ({len(similarities)}) kurang dari k ({k}). Mengembalikan semua sampel.")
        top_k_indices = np.arange(len(similarities))
    else:
        top_k_indices = np.argpartition(similarities, -k)[-k:]

    sorted_top_k_indices = top_k_indices[np.argsort(similarities[top_k_indices])][::-1]

    results = []
    original_valid_indices_mask = df_cases['text_pdf'].apply(lambda x: isinstance(x, str) and x.strip() != '')
    original_valid_indices = df_cases[original_valid_indices_mask].index.tolist()

    if len(original_valid_indices) != loaded_case_vectors.shape[0]:
        print(f"ERROR: Ketidakcocokan panjang antara original_valid_indices ({len(original_valid_indices)}) dan loaded_case_vectors ({loaded_case_vectors.shape[0]}).")
        print("Ini bisa terjadi jika ada dokumen yang dibuang setelah embedding atau kesalahan indexing.")
        return []

    for idx_in_corpus in sorted_top_k_indices:
        original_df_index = original_valid_indices[idx_in_corpus]
        case_info = df_cases.loc[original_df_index]
        similarity_score = similarities[idx_in_corpus]
        results.append({
            "case_id": case_info['nomor'],
            "judul": case_info['judul'],
            "klasifikasi": case_info['klasifikasi'],
            "similarity_score": float(similarity_score),
            "link": case_info['link']
        })

    print(f"Retrieval selesai. Ditemukan {len(results)} kasus.")
    return results

print("Fungsi utilitas dan pemuatan model Tahap 4 berhasil.")

Memuat model dan vektor dari Tahap 3...
Tipe vectorizer yang digunakan: BERT_Embedding
Memuat ulang SentenceTransformer model untuk BERT Embedding...
BERT embedding model dan case vectors berhasil dimuat.
Mencoba memuat cases.csv dari: /content/drive/MyDrive/CBR_Data/data/processed/cases.csv
Berhasil memuat 65 kasus dari cases.csv.
Fungsi utilitas dan pemuatan model Tahap 4 berhasil.


In [8]:
def extract_solution_text(case_row: pd.Series) -> str:
    """
    Mengekstrak teks "solusi" (amar putusan atau ringkasan dakwaan) dari baris kasus.
    Args:
        case_row (pd.Series): Baris (record) DataFrame dari kasus yang mirip.
    Returns:
        str: Teks solusi yang diekstrak.
    """
    # Prioritaskan 'amar' putusan
    if 'amar' in case_row and case_row['amar'] and case_row['amar'].strip() != 'nan':
        return f"Amar Putusan: {case_row['amar'].strip()}"
    # Jika 'amar' tidak ada atau kosong, coba 'catatan_amar'
    elif 'catatan_amar' in case_row and case_row['catatan_amar'] and case_row['catatan_amar'].strip() != 'nan':
        return f"Catatan Amar: {case_row['catatan_amar'].strip()}"
    # Fallback ke 'ringkasan_fakta' (yang mungkin berisi dakwaan atau ringkasan)
    elif 'ringkasan_fakta' in case_row and case_row['ringkasan_fakta'] and case_row['ringkasan_fakta'].strip() != 'nan':
        return f"Ringkasan Fakta (Kemungkinan Dakwaan): {case_row['ringkasan_fakta'].strip()}"
    # Fallback ke 'argumen_hukum_utama'
    elif 'argumen_hukum_utama' in case_row and case_row['argumen_hukum_utama'] and case_row['argumen_hukum_utama'].strip() != 'nan':
        return f"Argumen Hukum Utama: {case_row['argumen_hukum_utama'].strip()}"

    return "Solusi tidak dapat diekstrak dari kasus ini."

print("Fungsi ekstraksi solusi berhasil didefinisikan.")

Fungsi ekstraksi solusi berhasil didefinisikan.


In [10]:
import json # Pastikan json diimpor di sel ini juga jika belum

def predict_outcome(query: str, k: int = 5) -> Dict[str, Any]:
    """
    Memprediksi solusi untuk kasus baru berdasarkan top-k kasus terkemuka.
    Args:
        query (str): Teks query kasus baru.
        k (int): Jumlah kasus teratas yang akan dipertimbangkan.
    Returns:
        Dict[str, Any]: Dictionary berisi prediksi solusi dan top-k case IDs.
    """
    print(f"\nMemprediksi solusi untuk query: '{query[:50]}...'")

    # Dapatkan top-k kasus termirip menggunakan fungsi retrieve dari Tahap 3
    # Retrieve akan mengembalikan daftar dict: [{"case_id": ..., "judul": ..., "similarity_score": ...}]
    top_k_cases_retrieved = retrieve(query, k=k) # <<< SINTAKSIS SITASI DIHAPUS DI SINI

    if not top_k_cases_retrieved:
        print("Tidak ada kasus terkemuka yang ditemukan. Tidak dapat memprediksi solusi.")
        return {
            "predicted_solution": "Tidak dapat memprediksi solusi karena tidak ada kasus serupa ditemukan.",
            "top_k_case_ids": [],
            "top_k_details": []
        }

    solutions_with_similarity = []
    top_k_case_ids = []
    top_k_details = []

    for case_data in top_k_cases_retrieved:
        case_id = case_data['case_id']
        similarity_score = case_data['similarity_score']
        original_link = case_data['link']

        # Cari baris lengkap kasus di df_cases menggunakan case_id (nomor) atau link
        found_case_row = df_cases[df_cases['nomor'] == case_id]
        if found_case_row.empty:
             found_case_row = df_cases[df_cases['link'] == original_link]

        if not found_case_row.empty:
            solution_text = extract_solution_text(found_case_row.iloc[0])
            solutions_with_similarity.append({
                "solution": solution_text,
                "similarity": similarity_score
            })
            top_k_case_ids.append(case_id)
            top_k_details.append({
                "case_id": case_id,
                "judul": case_data['judul'],
                "klasifikasi": case_data['klasifikasi'],
                "similarity_score": similarity_score,
                "solution_text_preview": solution_text[:100] + "..." if len(solution_text) > 100 else solution_text
            })
        else:
            print(f"Peringatan: Detail kasus lengkap untuk case_id {case_id} tidak ditemukan di df_cases.")
            solutions_with_similarity.append({
                "solution": "Detail kasus tidak ditemukan.",
                "similarity": similarity_score
            })
            top_k_case_ids.append({
                "case_id": case_id,
                "judul": case_data['judul'],
                "klasifikasi": case_data['klasifikasi'],
                "similarity_score": similarity_score,
                "solution_text_preview": "Detail kasus tidak ditemukan."
            })


    predicted_solution = "Tidak dapat memprediksi solusi."
    if solutions_with_similarity:
        sorted_solutions = sorted(solutions_with_similarity, key=lambda x: x['similarity'], reverse=True)
        predicted_solution = sorted_solutions[0]['solution']
        print(f"DEBUG: Solusi prediksi diambil dari kasus dengan skor kemiripan tertinggi ({sorted_solutions[0]['similarity']:.4f}).")

    print("Prediksi solusi selesai.")
    return {
        "predicted_solution": predicted_solution,
        "top_k_case_ids": top_k_case_ids,
        "top_k_details": top_k_details
    }

print("Fungsi predict_outcome berhasil didefinisikan.")

Fungsi predict_outcome berhasil didefinisikan.


In [11]:
# --- Siapkan 5 contoh kasus baru untuk demo manual ---
sample_prediction_queries = [
    {
        "query_id": "PREDICT_Q1_Narkotika",
        "text": "Seorang mahasiswa ditangkap dengan barang bukti ekstasi 10 butir dan terjerat tindak pidana narkotika."
    },
    {
        "query_id": "PREDICT_Q2_Perdata",
        "text": "Kasus sengketa hutang piutang antara dua perusahaan yang berujung pada gugatan wanprestasi di pengadilan negeri."
    },
    {
        "query_id": "PREDICT_Q3_Pidana_Lain",
        "text": "Kasus penipuan online dengan kerugian jutaan rupiah melalui media sosial, pelaku menggunakan modus investasi bodong."
    }
]

# --- Output Path untuk hasil prediksi ---
base_drive_path = '/content/drive/MyDrive/CBR_Data' # Definisikan ulang base_drive_path
results_folder = create_path(os.path.join(base_drive_path, 'data/results'))
predictions_csv_path = os.path.join(results_folder, 'predictions.csv') # Hapus sintaks sitasi yang salah di sini

# --- Lakukan demo prediksi ---
print("\n--- Melakukan Demo Prediksi Solusi untuk Kasus Baru ---")
all_predictions_data = []

if df_cases.empty:
    print("Tidak dapat melakukan prediksi karena data kasus kosong. Pastikan Tahap 2 sudah berhasil.")
elif 'retrieve' not in locals() or not callable(retrieve):
    print("Fungsi 'retrieve' tidak ditemukan. Pastikan Sel 2 sudah dijalankan dengan benar.")
elif 'predict_outcome' not in locals() or not callable(predict_outcome):
    print("Fungsi 'predict_outcome' tidak ditemukan. Pastikan Sel 4 sudah dijalankan dengan benar.")
else:
    for query_data in sample_prediction_queries:
        query_text = query_data['text']
        query_id = query_data['query_id']

        predicted_output = predict_outcome(query_text, k=5)

        print(f"\nQuery ID: {query_id}")
        print(f"  Query Text: {query_text[:100]}...")
        print(f"  PREDIKSI SOLUSI: {predicted_output['predicted_solution'][:200]}...")
        print(f"  Top {len(predicted_output['top_k_case_ids'])} Kasus Termirip (IDs): {predicted_output['top_k_case_ids']}")

        all_predictions_data.append({
            "query_id": query_id,
            "predicted_solution": predicted_output['predicted_solution'],
            "top_5_case_ids": ", ".join(predicted_output['top_k_case_ids']),
            "top_5_details": json.dumps(predicted_output['top_k_details'], ensure_ascii=False)
        })

    if all_predictions_data:
        try:
            df_predictions = pd.DataFrame(all_predictions_data)
            df_predictions.to_csv(predictions_csv_path, index=False, encoding='utf-8')
            print(f"\nHasil prediksi berhasil disimpan ke: {predictions_csv_path}")
        except Exception as e:
            print(f"ERROR: Gagal menyimpan predictions.csv: {e}")
    else:
        print("Tidak ada data prediksi untuk disimpan.")


print("\nPROSES TAHAP 4 (SOLUTION REUSE) SELESAI.")

Folder '/content/drive/MyDrive/CBR_Data/data/results' sudah ada di Google Drive.

--- Melakukan Demo Prediksi Solusi untuk Kasus Baru ---

Memprediksi solusi untuk query: 'Seorang mahasiswa ditangkap dengan barang bukti ek...'

Melakukan retrieval untuk query: 'Seorang mahasiswa ditangkap dengan barang bukti ek...' (top-k=5)
Retrieval selesai. Ditemukan 5 kasus.
DEBUG: Solusi prediksi diambil dari kasus dengan skor kemiripan tertinggi (0.1897).
Prediksi solusi selesai.

Query ID: PREDICT_Q1_Narkotika
  Query Text: Seorang mahasiswa ditangkap dengan barang bukti ekstasi 10 butir dan terjerat tindak pidana narkotik...
  PREDIKSI SOLUSI: Amar Putusan: Lain-lain...
  Top 5 Kasus Termirip (IDs): ['154/PID.SUS/2025/PT MAM', '1109/PID.SUS/2025/PT MDN', '304/PID.SUS/2025/PT PBR', '172/PID.SUS/2025/PT TJK', '326/PID.SUS/2025/PT PBR']

Memprediksi solusi untuk query: 'Kasus sengketa hutang piutang antara dua perusahaa...'

Melakukan retrieval untuk query: 'Kasus sengketa hutang piutang antara du